In [2]:
import json
import os
from datetime import datetime
from utils.design_info import gen_design_info
from utils.git_info import gen_git_info
from utils.utils import gen_glue_module, write_blkinfo_to_dict, dump_jasper

In [3]:
# set env vars
os.environ['MLIB_DEVEL_PATH'] = '/home/wei/scilab_demo/mlib_devel'
os.environ['HDL_ROOT'] = '/home/wei/scilab_demo/mlib_devel/scilab_library/hdl_sources'

In [41]:
# get the scilab path from the environment variable
scilab_library_path = os.getenv('MLIB_DEVEL_PATH')+ '/scilab_library'

In [42]:
# open the jasper.json file
with open('../jasper.json') as f:
    blkinfo = json.load(f)
# after reading the file, remove it
# os.remove('jasper.json')
# open the block_info_template.json file
with open('%s/block_info_template.json'%(scilab_library_path)) as f:
    blkinfo_template = json.load(f)

In [43]:
# get the proj name
blkfn = blkinfo['project']['filename']
proj_name = blkfn.split('/')[-1].split('.')[0]
filepath = blkfn.split('.')[0]

In [44]:
# get the scilab blk, blk objs and link objs from the jaser.json file
scilab_blk_objs = []
blk_objs = []
link_info = blkinfo['link_info']
for k, v in blkinfo.items():
    if k.startswith('blk'):
        blk_objs.append(v)

In [45]:
print(len(blkinfo['link_info']))

7


In [46]:
# genenrate yellow blocks dict
xps_blocks = {}
for blk in blk_objs:
    tag = blk['tag']
    # check if the block in the blockinfo starts with xps
    if blkinfo_template[tag]['tag'].startswith('xps'):
        # populate yellow block
        key = proj_name + '/' + blk['val'][0][0]
        xps_blocks[key] = write_blkinfo_to_dict(blk, blkinfo_template[tag])
        xps_blocks[key]['fullpath'] = key

In [53]:
# generate xps_user_module dict
xps_user_modules = {}
# generate dsp_user_module dict
xps_user_modules = {}
for link in link_info:
    if link['link_type'] == 'xps_xps':
        gen_glue_module(link, file_dir=filepath)
        module_name = link['link_type'] + '_' + link['src_blk_name'] + '_' + link['dst_blk_name']
        xps_user_modules[module_name] = {}
        xps_user_modules[module_name]['clock'] = 'clk'
        # add ports
        xps_user_modules[module_name]['ports'] = []
        xps_user_modules[module_name]['ports'].append(link['src_port_name'])
        xps_user_modules[module_name]['ports'].append(link['dst_port_name'])
        # add sources
        xps_user_modules[module_name]['sources'] = []
        xps_user_modules[module_name]['sources'].append('%s/%s.v'%(filepath, module_name))  
# generate jasper.per
jasper_per = {}
jasper_per['yellow_blocks'] = xps_blocks
jasper_per['user_modules'] = xps_user_modules
#jasper_per_str = json.dumps(jasper_per, indent=2)
#print(jasper_per_str)
dump_jasper(jasper_per, fn='%s/jasper.per'%(filepath)) 

In [54]:
# now, we need to generate jasper.dsp, which contains the dsp block info
dsp_blocks = {}
for blk in blk_objs:
    tag = blk['tag']
    # check if the block in the blockinfo starts with dsp or xps:xsg
    # we need xps:xsg as it contains the platform info
    if blkinfo_template[tag]['tag'].startswith('dsp') or blkinfo_template[tag]['tag'] == 'xps:xsg':
        # populate dsp block
        key = proj_name + '/' + blk['val'][0][0]
        dsp_blocks[key] = write_blkinfo_to_dict(blk, blkinfo_template[tag])
        dsp_blocks[key]['fullpath'] = key
# generate dsp_user_module dict
dsp_user_modules = {}
for link in link_info:
    if link['link_type'] == 'dsp_dsp':
        gen_glue_module(link, file_dir=filepath)
        module_name = link['link_type'] + '_' + link['src_blk_name'] + '_' + link['dst_blk_name']
        dsp_user_modules[module_name] = {}
        dsp_user_modules[module_name]['clock'] = 'clk'
        # add ports
        dsp_user_modules[module_name]['ports'] = []
        dsp_user_modules[module_name]['ports'].append(link['src_port_name'])
        dsp_user_modules[module_name]['ports'].append(link['dst_port_name'])
        # add sources
        dsp_user_modules[module_name]['sources'] = []
        dsp_user_modules[module_name]['sources'].append('%s/%s.v'%(filepath, module_name))  
# generate jasper.dsp
jasper_dsp = {}
jasper_dsp['dsp_blocks'] = dsp_blocks
jasper_dsp['user_modules'] = dsp_user_modules
#jasper_dsp_str = json.dumps(jasper_dsp, indent=2)
#print(jasper_dsp_str)
dump_jasper(jasper_dsp, fn='%s/jasper.dsp'%(filepath))